In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 25171409
paper_name = 'jarosz_lindquist_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data1 = pd.read_excel('raw_data/mmc3.xls', sheet_name='Sheet1')
original_data2 = pd.read_excel('raw_data/mmc4.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data1.shape))
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 63 x 4
Original data dimensions: 32 x 4


In [7]:
original_data1['orf'] = original_data1['ORF name'].astype(str)
original_data2['orf'] = original_data2['ORF name'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data1['orf'] = clean_orf(original_data1['orf'])
original_data2['orf'] = clean_orf(original_data2['orf'])

In [9]:
# Translate to ORFs 
original_data1['orf'] = translate_sc(original_data1['orf'], to='orf')
original_data2['orf'] = translate_sc(original_data2['orf'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['orf'])
print(original_data1.loc[~t,])

                          ORF name Gene name Alias Gene description  \
index_input                                                           
61                             NaN       NaN   NaN              NaN   
62           Supplementary Table 2       NaN   NaN              NaN   

                             orf  
index_input                       
61                           NAN  
62           SUPPLEMENTARYTABLE2  


In [11]:
original_data1.drop(index=original_data1.loc[~t].index, inplace=True)

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orf'])
print(original_data2.loc[~t,])

                          ORF name Gene name Alias Gene description  \
index_input                                                           
30                             NaN       NaN   NaN              NaN   
31           Supplementary Table 5       NaN   NaN              NaN   

                             orf  
index_input                       
30                           NAN  
31           SUPPLEMENTARYTABLE5  


In [13]:
original_data2.drop(index=original_data2.loc[~t].index, inplace=True)

In [14]:
original_data1['data'] = -1
original_data2['data'] = 1

In [15]:
original_data1.set_index('orf', inplace=True)
original_data2.set_index('orf', inplace=True)

In [16]:
original_data1 = original_data1[['data']].copy()
original_data2 = original_data2[['data']].copy()

In [17]:
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data2 = original_data2.groupby(original_data2.index).mean()

In [20]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [21]:
original_data.head()

,data_1,data_2
orf,,
YAL020C,-1.0,NaN
YAL037W,-1.0,NaN
YAL039C,-1.0,NaN
YAL047C,NaN,1.0
YBL090W,-1.0,NaN


In [22]:
original_data['data'] = original_data.mean(axis=1)

In [23]:
original_data.drop(columns=['data_1','data_2'], inplace=True)

# Prepare the final dataset

In [24]:
data = original_data.copy()

In [25]:
dataset_ids = [16510]
datasets = datasets.reindex(index=dataset_ids)

In [26]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [27]:
data.head()

dataset_id,16510
data_type,value
orf,
YAL020C,-1.0
YAL037W,-1.0
YAL039C,-1.0
YAL047C,1.0
YBL090W,-1.0


## Subset to the genes currently in SGD

In [28]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [29]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16510
,data_type,value
gene_id,orf,
18,YAL020C,-1.0
35,YAL037W,-1.0
37,YAL039C,-1.0
45,YAL047C,1.0
178,YBL090W,-1.0


# Normalize

In [30]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [31]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [32]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      16510          
data_type       value    valuez
gene_id orf                    
18      YAL020C  -1.0 -7.015068
35      YAL037W  -1.0 -7.015068
37      YAL039C  -1.0 -7.015068
45      YAL047C   1.0  7.158232
178     YBL090W  -1.0 -7.015068

# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [36]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 25171409...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.85it/s]

Updating the data_modified_on field...
